In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\2\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\2\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[45.0, 95.0, 20.0, 13.0, 93.0, 45.0, 20.0, 40.0, 13.0, 40.0, 16.0, 47.0, 20.0, 45.0, 16.0, 45.0, 93.0, 10.0, 37.0, 86.0, 37.0, 37.0, 30.0, 13.0, 16.0, 7.0, 80.0, 66.0, 80.0, 30.0, 86.0, 95.0, 37.0, 7.0, 45.0, 96.0, 7.0, 39.0, 7.0, 66.0, 47.0, 37.0, 100.0, 66.0, 86.0, 47.0, 47.0, 95.0, 23.0, 100.0, 45.0, 45.0, 32.0, 4.0, 16.0, 55.0, 96.0, 7.0, 55.0, 80.0, 86.0, 32.0, 95.0, 16.0, 47.0, 7.0, 80.0, 95.0, 7.0, 37.0, 13.0, 66.0, 4.0, 35.0, 32.0, 84.0, 7.0, 37.0, 47.0, 32.0, 35.0, 40.0, 93.0, 16.0, 37.0, 40.0, 7.0, 7.0, 100.0, 45.0, 7.0, 47.0, 47.0, 93.0, 84.0, 37.0, 66.0, 84.0, 13.0, 40.0, 7.0, 37.0, 100.0, 37.0, 16.0, 40.0, 86.0, 32.0, 95.0, 52.0, 37.0, 37.0, 95.0, 39.0, 30.0, 16.0, 37.0, 47.0, 13.0, 52.0, 23.0, 95.0, 40.0, 80.0, 13.0, 7.0, 95.0, 7.0, 13.0, 52.0, 37.0, 13.0, 84.0, 96.0, 37.0, 37.0, 100.0, 95.0, 35.0, 84.0, 86.0, 96.0, 84.0, 100.0, 93.0, 96.0, 32.0, 39.0, 45.0, 100.0, 80.0, 55.0, 40.0, 47.0, 37.0, 10.0, 40.0, 95.0, 32.0, 100.0, 7.0, 45.0, 66.0, 52.0, 37.0, 35.0, 80.0, 93.0, 

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [6]:
#要改
RegNetY002 =tf.keras.applications.regnet.RegNetY002(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY002.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY002(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety002 (Functional)     (None, 7, 7, 368)         2814844   
                                                                 
 global_average_pooling2d (G  (None, 368)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 369       
                                                                 
Total params: 2,815,213
Trainable params: 2,794,365
Non-trainable params: 20,848
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY002_loss_2.txt',history.history['loss'])
#要改
np.savetxt('RegNetY002_valloss_2.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Thu Apr 13 16:00:28 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 891.2175

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


591/591 [==============================] - 73s 101ms/step - loss: 891.2175 - val_loss: 67.2400
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 44.6254

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


591/591 [==============================] - 119s 201ms/step - loss: 1.6592 - val_loss: 0.4926
Epoch 266/300
591/591 [==============================] - 100s 169ms/step - loss: 1.5322 - val_loss: 0.6370
Epoch 267/300
591/591 [==============================] - 99s 168ms/step - loss: 1.8705 - val_loss: 0.7680
Epoch 268/300
591/591 [==============================] - 100s 169ms/step - loss: 1.5592 - val_loss: 0.8681
Epoch 269/300
591/591 [==============================] - 98s 166ms/step - loss: 1.5322 - val_loss: 0.5372
Epoch 270/300
591/591 [==============================] - 92s 155ms/step - loss: 1.5782 - val_loss: 0.5203
Epoch 271/300
591/591 [==============================] - ETA: 0s - loss: 1.6108

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY002\2\RegNetY002.ckpt\assets


591/591 [==============================] - 112s 189ms/step - loss: 1.6108 - val_loss: 0.4828
Epoch 272/300
591/591 [==============================] - 90s 153ms/step - loss: 1.5477 - val_loss: 0.5985
Epoch 273/300
591/591 [==============================] - 89s 151ms/step - loss: 1.4761 - val_loss: 0.4976
Epoch 274/300
591/591 [==============================] - 90s 152ms/step - loss: 1.6426 - val_loss: 0.6777
Epoch 275/300
591/591 [==============================] - 89s 151ms/step - loss: 1.9699 - val_loss: 0.5993
Epoch 276/300
591/591 [==============================] - 90s 151ms/step - loss: 1.4740 - val_loss: 0.7542
Epoch 277/300
591/591 [==============================] - 88s 148ms/step - loss: 1.6301 - val_loss: 0.5108
Epoch 278/300
591/591 [==============================] - 91s 154ms/step - loss: 1.4885 - val_loss: 0.5379
Epoch 279/300
591/591 [==============================] - 90s 152ms/step - loss: 1.5986 - val_loss: 0.5524
Epoch 280/300
591/591 [==============================] - 90